In [17]:
import tkinter as tk
from tkinter import messagebox
import pandas as pd
import joblib
from sklearn.preprocessing import StandardScaler

# Wczytanie modelu i scalera
model_path = r"C:\Users\damia\Desktop\Przewidywanie_global\best_model.pkl"
scaler_path = r"C:\Users\damia\Desktop\Przewidywanie_global\scaler.pkl"
model = joblib.load(model_path)
scaler = joblib.load(scaler_path)

# Lista cech do skalowania
features_to_scale = ['Prognoza', 'Palety załadowane 6:00 do 6:00', 'Srednia_3dni', 'Trend', 'Srednia_7dni']

# Funkcja do obliczania średnich i trendu
def calculate_features(data):
    df = pd.DataFrame(data)
    
    # Obliczenie średnich
    df['Srednia_3dni'] = df['Real'].rolling(window=3).mean().shift(1)
    df['Srednia_7dni'] = df['Real'].rolling(window=7).mean().shift(1)
    
    # Obliczenie trendu
    df['Trend'] = df['Real'] - df['Srednia_7dni']
    
    # Obliczenie odchylenia
    df['Odchylenie'] = df['Real'] - df['Prognoza']
    
    # Wypełnianie braków wartościami zastępczymi (np. zerami lub inną strategią)
    df['Srednia_3dni'].fillna(df['Real'].mean(), inplace=True)
    df['Srednia_7dni'].fillna(df['Real'].mean(), inplace=True)
    df['Trend'].fillna(0, inplace=True)
    
    return df

# Funkcja do walidacji wpisów użytkownika
def validate_entries(entries):
    for entry in entries:
        if not entry.get().strip() or not entry.get().replace('.', '', 1).isdigit():
            return False
    return True

# Funkcja do przewidywania wartości Real
def predict_real():
    try:
        # Zbieranie danych od użytkownika
        data = []
        
        # Walidacja wszystkich wpisów
        if not validate_entries(real_entries) or not validate_entries(prognoza_entries) or not validate_entries(palety_entries):
            raise ValueError("Proszę wprowadzić wszystkie wartości poprawnie.")
        
        for i in range(7):
            real = float(real_entries[i].get())
            prognoza = float(prognoza_entries[i].get())
            palety = float(palety_entries[i].get())
            data.append({'Real': real, 'Prognoza': prognoza, 'Palety załadowane 6:00 do 6:00': palety})
        
        # Obliczanie funkcji inżynieryjnych
        df = calculate_features(data)
        
        # Dodanie kolumny Data na podstawie wybranego dnia
        try:
            selected_date = pd.to_datetime(entry_date.get(), format='%Y-%m-%d')
        except ValueError:
            raise ValueError("Proszę wprowadzić poprawną datę w formacie YYYY-MM-DD.")
        
        df['Data'] = pd.date_range(end=selected_date, periods=7)
        
        # Dodanie kolumn 'Dzien', 'Rok', 'Dzien_tygodnia', 'Miesiac', 'Czy_weekend'
        df['Dzien'] = df['Data'].dt.day
        df['Rok'] = df['Data'].dt.year
        df['Dzien_tygodnia'] = df['Data'].dt.dayofweek
        df['Miesiac'] = df['Data'].dt.month
        df['Czy_weekend'] = df['Dzien_tygodnia'].apply(lambda x: 1 if x >= 5 else 0)

        # Wybieramy ostatni dzień, dla którego chcemy przewidzieć Real
        X_pred = df.drop(columns=['Data', 'Real', 'Odchylenie']).iloc[-1:]
        
        # Kolejność cech zgodnie z modelem (aktualizacja na podstawie tego, co model oczekuje)
        feature_names = ['Prognoza', 'Palety załadowane 6:00 do 6:00', 'Dzien_tygodnia', 'Miesiac',
                         'Czy_weekend', 'Srednia_3dni', 'Trend', 'Srednia_7dni', 'Dzien', 'Rok']
        
        # Ustalmy kolejność cech zgodnie z modelem
        X_pred = X_pred[feature_names]

        # Debugowanie - wyświetlenie nagłówków i ich kolejności
        print("Nagłówki X_pred:", X_pred.columns.tolist())
        
        # Debugowanie - wyświetlenie zawartości X_pred
        print("Zawartość X_pred:\n", X_pred)

        # Upewnijmy się, że kolumny X_pred są numeryczne i że nie ma brakujących wartości
        if X_pred.isnull().any().any():
            raise ValueError("Wystąpiły braki danych. Upewnij się, że wszystkie pola są wypełnione.")
        
        # Skalowanie danych przed predykcją
        X_pred[features_to_scale] = scaler.transform(X_pred[features_to_scale])
        
        # Przewidywanie wartości Real
        pred_real = model.predict(X_pred)[0]
        
        # Wyświetlanie wyniku
        label_result.config(text=f"Przewidywana wartość Real: {pred_real:.2f}")
    
    except Exception as e:
        messagebox.showerror("Błąd", str(e))

# Interfejs użytkownika
root = tk.Tk()
root.title("Predykcja wartości Real")

# Pola do wprowadzania danych z 7 dni
real_entries = []
prognoza_entries = []
palety_entries = []

for i in range(7):
    tk.Label(root, text=f"Dzień {i+1} - Real:").grid(row=i, column=0)
    real_entry = tk.Entry(root)
    real_entry.grid(row=i, column=1)
    real_entries.append(real_entry)
    
    tk.Label(root, text=f"Prognoza:").grid(row=i, column=2)
    prognoza_entry = tk.Entry(root)
    prognoza_entry.grid(row=i, column=3)
    prognoza_entries.append(prognoza_entry)
    
    tk.Label(root, text=f"Palety załadowane 6:00 do 6:00:").grid(row=i, column=4)
    palety_entry = tk.Entry(root)
    palety_entry.grid(row=i, column=5)
    palety_entries.append(palety_entry)

# Pole do wprowadzenia daty, dla której przewidujemy Real
tk.Label(root, text="Data (YYYY-MM-DD):").grid(row=7, column=0)
entry_date = tk.Entry(root)
entry_date.grid(row=7, column=1)

# Przycisk do przewidywania
predict_button = tk.Button(root, text="Przewidź Real", command=predict_real)
predict_button.grid(row=8, column=0, columnspan=2)

# Etykieta do wyświetlania wyniku
label_result = tk.Label(root, text="")
label_result.grid(row=9, column=0, columnspan=6)

root.mainloop()

Nagłówki X_pred: ['Prognoza', 'Palety załadowane 6:00 do 6:00', 'Dzien_tygodnia', 'Miesiac', 'Czy_weekend', 'Srednia_3dni', 'Trend', 'Srednia_7dni', 'Dzien', 'Rok']
Zawartość X_pred:
     Prognoza  Palety załadowane 6:00 do 6:00  Dzien_tygodnia  Miesiac  \
6  4141414.0                       1414141.0               3        8   

   Czy_weekend   Srednia_3dni  Trend   Srednia_7dni  Dzien   Rok  
6            0  476565.333333    0.0  963858.142857      1  2024  
